In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from utils import generate_groups, generate_points_and_distances, compute_minimum_distances, compute_medical_deserts, plot_state, plot_facilities
from facility_location import facility_location, round_lp_solution
import os
from src.usa.states import USAState
from src.usa.constants import state_names
import csv
from tqdm import tqdm
from pprint import pprint
import sys
from ast import literal_eval
import random

from src.constants import scatter_palette


racial_labels = ['white_alone', 'black_alone', 'aian_alone',
                 'asian_alone', 'nhopi_alone', 'hispanic', 'other']
colors = {racial_labels[i]: scatter_palette[i] for i in range(len(racial_labels))}



In [14]:
facility = 'private_schools'

new_facility_1_all_states = pd.read_csv('../data/usa/new_facilities/' + facility + '/new_facilities_1.csv', index_col=0)['100'].apply(literal_eval)
new_facility_2_all_states = pd.read_csv('../data/usa/new_facilities/' + facility + '/new_facilities_2.csv', index_col=0)['100'].apply(literal_eval)
new_facility_inf_all_states = pd.read_csv('../data/usa/new_facilities/' + facility + '/new_facilities_inf.csv', index_col=0)['100'].apply(literal_eval)

for state in tqdm(state_names):
    # print(state, file=sys.stderr)
    new_facilities_1 = new_facility_1_all_states[state]
    new_facilities_2 = new_facility_2_all_states[state]
    new_facilities_inf = new_facility_inf_all_states[state]

    State = USAState(state)
    state_df = State.get_census_data(level='blockgroup')

    existing_distances_label = 'closest_distance_' + facility

    for k in [5, 10, 25, 50, 100]:
        facilities_1 = new_facilities_1[:k]
        facilities_2 = new_facilities_2[:k]
        facilities_inf = new_facilities_inf[:k]
        
        new_distance_label = existing_distances_label + '_p_1_k_' + str(k)
        state_df = compute_minimum_distances(state_df, facilities_1, existing_distances_label, new_distance_label)
        
        new_distance_label = existing_distances_label + '_p_2_k_' + str(k)
        state_df = compute_minimum_distances(state_df, facilities_2, existing_distances_label, new_distance_label)
        
        new_distance_label = existing_distances_label + '_p_inf_k_' + str(k)
        state_df = compute_minimum_distances(state_df, facilities_inf, existing_distances_label, new_distance_label)
        
    state_df.to_csv(f'../data/usa/census/' + State.fips + '/census_blockgroup_data.csv', index=False)
        

100%|██████████| 50/50 [14:44<00:00, 17.70s/it]


In [12]:
state_df

,State,County,Tract,Block_group,GEOID,population,white_alone,black_alone,aian_alone,asian_alone,...,closest_distance_private_schools_p_inf_k_10,closest_distance_private_schools_p_1_k_25,closest_distance_private_schools_p_2_k_25,closest_distance_private_schools_p_inf_k_25,closest_distance_private_schools_p_1_k_50,closest_distance_private_schools_p_2_k_50,closest_distance_private_schools_p_inf_k_50,closest_distance_private_schools_p_1_k_100,closest_distance_private_schools_p_2_k_100,closest_distance_private_schools_p_inf_k_100
0,13,1,950100,1,130019501001,1415,77.31,6.86,0.00,0.00,...,0.000,15.957,15.957,0.000,15.957,15.957,0.000,15.957,15.957,0.000
1,13,1,950100,2,130019501002,216,100.00,0.00,0.00,0.00,...,21.548,21.548,21.548,21.548,21.548,21.548,21.548,21.548,21.548,21.548
2,13,1,950100,3,130019501003,1638,93.59,4.15,0.00,0.00,...,9.751,16.137,16.137,9.751,16.137,16.137,9.751,16.137,16.137,9.751
3,13,1,950201,1,130019502011,1042,52.30,26.30,10.65,0.00,...,10.250,10.250,10.250,10.250,10.250,10.250,10.250,10.250,10.250,10.250
4,13,1,950201,2,130019502012,1234,57.86,29.42,0.00,0.00,...,8.277,8.277,8.277,8.277,8.277,8.277,8.277,8.277,8.277,8.277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7426,13,321,950500,2,133219505002,948,6.75,80.70,0.00,0.00,...,26.502,26.502,26.502,2.634,26.502,26.502,2.634,2.634,24.883,2.634
7427,13,321,950500,3,133219505003,643,90.82,0.00,0.00,3.27,...,12.144,12.144,12.144,12.144,12.144,12.144,12.144,12.144,12.144,12.144
7428,13,321,950500,4,133219505004,1031,51.79,41.51,0.00,5.82,...,17.504,17.504,17.504,9.459,17.504,17.504,9.459,9.459,17.504,9.459
7429,13,321,950600,1,133219506001,1214,84.27,4.94,0.25,3.71,...,15.788,15.788,15.788,15.788,15.788,15.788,15.788,15.788,15.788,15.788
